# How to use a SmartLLMChain

A SmartLLMChain is a form of self-critique chain that can help you if have particularly complex questions to answer. Instead of doing a single LLM pass, it instead performs these 3 steps:
1. Ideation: Pass the user prompt n times through the LLM to get n output proposals (called "ideas"), where n is a parameter you can set 
2. Critique: The LLM critiques all ideas to find possible flaws and picks the best one 
3. Resolve: The LLM tries to improve upon the best idea (as chosen in the critique step) and outputs it. This is then the final output.

SmartLLMChains are based on the SmartGPT workflow proposed in https://youtu.be/wVzuvf9D9BU.

Note that SmartLLMChains
- use more LLM passes (ie n+2 instead of just 1)
- only work then the underlying LLM has the capability for reflection, which smaller models often don't
- only work with underlying models that return exactly 1 output, not multiple

This notebook demonstrates how to use a SmartLLMChain.

##### Same LLM for all steps

In [2]:
from langchain.prompts import PromptTemplate
from langchain_experimental.smart_llm import SmartLLMChain
from langchain_community.chat_models import ChatOllama
model = ChatOllama(model="qwen:32b-chat-v1.5-q5_K_M", temperature=0)

As example question, we will use "I have a 12 liter jug and a 6 liter jug. I want to measure 6 liters. How do I do it?". This is an example from the original SmartGPT video (https://youtu.be/wVzuvf9D9BU?t=384). While this seems like a very easy question, LLMs struggle do these kinds of questions that involve numbers and physical reasoning.

As we will see, all 3 initial ideas are completely wrong - even though we're using GPT4! Only when using self-reflection do we get a correct answer. 

In [8]:
hard_question = "我有一个12升的罐子和一个6升的罐子。我要测量6升。我该怎么做呢?"
prompt = PromptTemplate.from_template(hard_question)
chain = SmartLLMChain(llm=model, prompt=prompt, n_ideas=3, verbose=True)

So, we first create an LLM and prompt template

Now we can create a SmartLLMChain

Now we can use the SmartLLM as a drop-in replacement for our LLM. E.g.:

In [9]:
output = chain.invoke({})

for o in output:
    print(output)



> Entering new SmartLLMChain chain...
Prompt after formatting:
我有一个12升的罐子和一个6升的罐子。我要测量6升。我该怎么做呢?
Idea 1:
1. 首先，将12升的罐子装满水。
2. 然后，从12升的罐子里倒出6升水到6升的空罐子里，这样12升的罐子里就剩下6升水了。
3. 把这6升水倒入空的12升罐子中，此时12升罐子已满，留下6升的空间。
4. 再次将12升的罐子装满水。
5. 现在，从12升的罐子里倒出水到已经装有6升水的6升罐子中，直到6升罐子满。由于它已经有6升，你只需要再加6升 - 6升 = 0升，但在这个过程中，你会从12升的罐子里溢出正好6升的水。

这样，你就成功地使用了这两个罐子测量出了准确的6升水。
Idea 2:
1. 首先，将12升的罐子装满水。
2. 然后，从12升的罐子里倒出6升水到6升的空罐子里，这样12升的罐子里就剩下6升水了。
3. 把这6升水倒入另一个容器或者放回水源，现在12升的罐子是空的。
4. 再次将12升的罐子装满水。
5. 从12升的罐子里倒出水到6升的罐子，直到6升的罐子满为止。由于你已经知道如何用12升罐子和6升罐子得到恰好6升的水（步骤2），所以你会在正确的时候停止倒水。

这样你就成功地使用了这两个罐子测量出了6升水。
Idea 3:
1. 首先，将12升的罐子装满水。
2. 然后，从12升的罐子里倒出6升水到6升的空罐子里，这样12升的罐子里就剩下6升水了。
3. 把这6升水倒入另一个容器或者放回水源，现在12升的罐子是空的。
4. 再次将12升的罐子装满水。
5. 从12升的罐子里倒出水到6升的罐子，直到6升的罐子满为止。由于你已经知道如何用12升罐子和6升罐子得到恰好6升的水（步骤2），所以你会在正确的时候停止倒水。

这样你就成功地使用了这两个罐子测量出了6升水。
Critique:
I'm sorry, but I don't see the three response options you mentioned. Could you please provide me with those options so that I can help you identify the flaws and faulty logic in

##### Different LLM for different steps

You can also use different LLMs for the different steps by passing `ideation_llm`, `critique_llm` and `resolve_llm`. You might want to do this to use a more creative (i.e., high-temperature) model for ideation and a more strict (i.e., low-temperature) model for critique and resolution.

In [8]:
chain = SmartLLMChain(
    ideation_llm=ChatOpenAI(temperature=0.9, model_name="gpt-4"),
    llm=ChatOpenAI(
        temperature=0, model_name="gpt-4"
    ),  # will be used for critique and resolution as no specific llms are given
    prompt=prompt,
    n_ideas=3,
    verbose=True,
)